In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w6p2/X_test.csv
/kaggle/input/2022-ai-w6p2/train.csv
/kaggle/input/2022-ai-w6p2/sample_submit.csv


In [3]:
train = pd.read_csv('../input/2022-ai-w6p2/train.csv')
test = pd.read_csv('../input/2022-ai-w6p2/X_test.csv')
submit = pd.read_csv('../input/2022-ai-w6p2/sample_submit.csv')

In [4]:
import random
import torch
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [5]:
trainX = train.drop(['Day', 'Minute', 'TARGET'], axis=1)
trainY = train['TARGET']
testX = test.drop(['ID', 'Day', 'Minute'], axis=1)

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
trainX = scaler.fit_transform(trainX)
testX = scaler.transform(testX)

In [7]:
x_train = torch.FloatTensor(trainX).cuda()
y_train = torch.FloatTensor(trainY).cuda()
x_test = torch.FloatTensor(testX).cuda()

In [8]:
x_train.shape

torch.Size([9999, 6])

In [10]:
y_train

tensor([ 0.0000,  0.0000,  0.0000,  ..., 13.8872, 23.2699, 33.0276],
       device='cuda:0')

In [11]:
# NN 모델
n = x_train.shape[1]


layer1 = torch.nn.Linear(n, 16, bias=True).cuda()
layer2 = torch.nn.Linear(16, 16, bias=True).cuda()
layer3 = torch.nn.Linear(16, 1, bias=True).cuda()
sigmoid = torch.nn.Sigmoid()

In [12]:
model = torch.nn.Sequential(layer1, sigmoid, layer2, sigmoid, layer3)

In [14]:
# 모델학습 환경 설정
loss = torch.nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1)

In [18]:
y_train.shape

torch.Size([9999])

In [21]:
y_train.reshape(-1,1).shape

torch.Size([9999, 1])

In [22]:
import torch.nn.functional as F

np_epochs = 10000
for epoch in range(np_epochs+1) :
    # gradient 초기화
    optimizer.zero_grad()
    # forward 계산
    hypothesis = model(x_train)
    # error 계산
    cost = torch.sqrt(loss(hypothesis, y_train.reshape(-1,1)))
    # backward 계산
    cost.backward()
    # 가중치 계산
    optimizer.step()
    
    if epoch % 1000 == 0:
        print(epoch, cost.item())
    

0 26.2175235748291
1000 3.0524559020996094
2000 2.349012851715088
3000 1.9784728288650513
4000 1.756424069404602
5000 1.6053441762924194
6000 1.4949580430984497
7000 1.420468807220459
8000 1.3712867498397827
9000 1.3352351188659668
10000 1.3057869672775269


In [24]:
with torch.no_grad() :
    hypothesis = model(x_test)


In [26]:
submit['TARGET'] = torch.Tensor.cpu(hypothesis)

In [27]:
submit.to_csv('submit.csv', index=False)